<a href="https://colab.research.google.com/github/sghuffar/Glacial_Lakes_Sentinel/blob/master/Sentinel_Lakes_Delineation_DeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a code to segment a Sentinel-2 image in to water and non water binary image. The segmentation is performed using the U-Net convolutional Neural Network model trained using the Pytorch Library. 
Part of the code is based on the following implementation: Deep networks for Earth Observation (https://github.com/nshaud/DeepNetsForEO). I would like to hereby acknowledge that the above mentioned respository has helped me alot in implementation. I am grateful to the authors for sharing their code. 

In order to run this code, a Google account is required. The Sentinel-2 image is exported from the EarthEngine to the Drive and then feeded in to the neural network for prediction. The exporting of the image to the Google Drive may take 10-20 minutes. Large size may crash colab

The False color original image as well as the predicted mask is then displayed on the map. 

The user needs to auhtorize access to the Google Drive below


In [1]:
!pip install pyproj   # pyproj is required for coordinate transformations
!pip install rasterio 
import rasterio as rio
import torch
print(torch.__version__)
import torch.nn.functional as F
from skimage import io
from glob import glob
import folium   # for visualizing maps and images
import random
import itertools
import gdal
import matplotlib.pyplot as plt
import os.path
from torch.autograd import Variable
import numpy as np
import os
import torch.nn as nn
from torchvision import transforms  
import torch.utils.data as data
import torch.optim as optim
import torch.optim.lr_scheduler
import torch.nn.init
from tqdm import tqdm
import numpy as np
from pyproj import Proj, transform
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')
    
use_cuda = True

from google.colab import drive
drive.mount('/content/gdrive')

     |████████████████████████████████| 10.9MB 277kB/s 
     |████████████████████████████████| 18.2MB 54.8MB/s 
1.6.0+cu101
CUDA is available!  Training on GPU ...
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


As we are using Earth Engine libraries to read and export the Sentinel-2 image, one would need to authorize access for Earth Engine. Copy paste the code below for authorization. 

In [3]:
!pip install earthengine-api  # install the Earth Engine API
!earthengine authenticate
import ee
ee.Initialize()


Instructions for updating:
non-resource variables are not supported in the long term
Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=mJiIDzJHXcLE-ZRj7WCEB9-iHIdauBlt7B106jqBmBo&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/2wG1QgYMFuG_ocwOB8-p9HYmDFg5XlbD5NmrjfKhyEAE243pQocFl3g

Successfully saved authorization token.


Below we select a Sentinel-2 image. At the moment only a central region of the image is exported because exporting a full image takes quite long time

In [7]:
image=ee.Image('COPERNICUS/S2/20180806T053641_20180806T054709_T43SFV')  #change the name of this image if testing on another image is required
projI=image.select('B2').projection().getInfo()    # Exporting the whole image takes time, therefore, roughly select the center region of the image
metadata=image.select('B2').getInfo()
print(metadata)
dim=metadata.get('bands')[0]
print(dim)
dim=dim.get('dimensions')
print(dim)

crs=projI['crs']
outProj=Proj(init='epsg:4326')
inProj=Proj(init=crs)
R=projI['transform']
print(projI)
print(R)
#lon1, lat1=transform(inProj,outProj,R[2],R[5])    # corner of the Sentinel-2 image using the Georeferencing Matrix R
#lon2, lat2=transform(inProj,outProj,R[2]+dim[0]*10,R[5]-dim[1]*10)  

lon1, lat1=transform(inProj,outProj,R[2]+(2000*10),R[5]-(2000*10))
lon2, lat2=transform(inProj,outProj,R[2]+(5000*10),R[5]-(5000*10)) #change these coordinates to increase or decrease image size

bounds = [lon1, lat2, lon2, lat1]
print(bounds,lon1,lat1)

imageS2=image.select(['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B10','B11','B12'])

geometry = ([lon1, lat1],[lon1,lat2],[lon2,lat2],[lon2 ,lat1])
print(geometry)
config= {
    'description':'20180806T053641_20180806T054709_T43SFV',    
    'region':geometry ,
    'scale': 10,  #the image is exported with 15m resolution
    'fileFormat': 'GeoTIFF',
    'maxPixels':'10000000000000'
}
exp=ee.batch.Export.image.toDrive(imageS2,**config);

exp.start()   # It takes around 5-10 minutes for 6000 * 6000 * 8 image to be exported 
print(exp.status())
print(ee.batch.Task.list())
import time
while exp.active():
  print('Transferring Data to Drive..................')
  time.sleep(30)
print('Done with the Export to the Drive')

{'type': 'Image', 'bands': [{'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32643', 'crs_transform': [10, 0, 600000, 0, -10, 4000020]}], 'id': 'COPERNICUS/S2/20180806T053641_20180806T054709_T43SFV', 'version': 1534627439370661, 'properties': {'DATATAKE_IDENTIFIER': 'GS2A_20180806T053641_016304_N02.06', 'SPACECRAFT_NAME': 'Sentinel-2A', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A': 217.650662986, 'MEAN_SOLAR_AZIMUTH_ANGLE': 136.06640453, 'system:footprint': {'type': 'LinearRing', 'coordinates': [[76.11137015041298, 36.13975304210107], [76.11136917703857, 36.13974179058083], [76.10447468069863, 35.644940487022964], [76.0977462120155, 35.150097321980795], [76.09779037220943, 35.15005548173873], [76.09782780659111, 35.15000940176183], [76.09784586004814, 35.1500064857658], [77.30244104651541, 35.13310073708927], [77.30249276394431, 35.13313636023269], [77.3025495654331, 35.133166329525906], [77.3025534223233, 35.

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axi

{'state': 'READY', 'description': '20180806T053641_20180806T054709_T43SFV', 'creation_timestamp_ms': 1597329574522, 'update_timestamp_ms': 1597329574522, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'HFAFT3HVORNLHTOVM42CSZ6E', 'name': 'projects/earthengine-legacy/operations/HFAFT3HVORNLHTOVM42CSZ6E'}
[<Task EXPORT_IMAGE: 20180806T053641_20180806T054709_T43SFV (READY)>, <Task EXPORT_IMAGE: 20180806T053641_20180806T054709_T43SFV (COMPLETED)>, <Task EXPORT_IMAGE: 20180806T053641_20180806T054709_T43SFV (COMPLETED)>, <Task EXPORT_IMAGE: 20180806T053641_20180806T054709_T43SFV (COMPLETED)>, <Task EXPORT_IMAGE: Sentinel_20180808_Baltoro (COMPLETED)>, <Task EXPORT_IMAGE: Sentinel_20180808_Baltoro (COMPLETED)>]
Transferring Data to Drive..................
Transferring Data to Drive..................
Transferring Data to Drive..................
Transferring Data to Drive..................
Transferring Data to Drive..................
Transferring Data to Drive..................
Tran

This portion contains the network definition. There are five encoder and five decoder units. 

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvBnRelu(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding, stride):
        super(ConvBnRelu, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding, stride=stride,padding_mode='replicate')
        self.bn = nn.BatchNorm2d(in_channels)
        #self.bn = nn.GroupNorm(in_channels,in_channels)
        #self.relu = nn.ReLU()
        self.relu = nn.LeakyReLU()

    def forward(self, x): 
        x = self.bn(x)
        x = self.conv(x)       
        x = self.relu(x)        
        return x

class StackEncoder(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(StackEncoder, self).__init__()
        self.convr1 = ConvBnRelu(in_channels, out_channels, kernel_size=(3, 3), stride=1, padding=1)
        self.convr2 = ConvBnRelu(out_channels, out_channels, kernel_size=(3, 3), stride=1, padding=1)
        self.maxPool = nn.MaxPool2d(kernel_size=(2, 2), stride=2)

    def forward(self, x):
        x = self.convr1(x)
        x = self.convr2(x)
        x_trace = x
        x = self.maxPool(x)
        return x, x_trace

class StackEncoder2(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(StackEncoder2, self).__init__()
        self.convr1 = ConvBnRelu(in_channels, out_channels, kernel_size=(3, 3), stride=1, padding=1)
        self.convr2 = ConvBnRelu(out_channels, out_channels, kernel_size=(3, 3), stride=1, padding=1)
        self.convr3 = ConvBnRelu(out_channels, out_channels, kernel_size=(3, 3), stride=1, padding=1)
        self.maxPool = nn.MaxPool2d(kernel_size=(2, 2), stride=2)

    def forward(self, x):
        x = self.convr1(x)
        x = self.convr2(x)
        x = self.convr3(x)
       
        x_trace = x
        x = self.maxPool(x)
        return x, x_trace

class StackDecoder(nn.Module):
    def __init__(self, in_channels1,in_channels2, out_channels):
        super(StackDecoder, self).__init__()
        self.upSample = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)
        #self.upSample = nn.ConvTranspose2d(in_channels1,in_channels1, (2,2), stride=2)
        self.convr1 = ConvBnRelu(in_channels1+in_channels2, out_channels, kernel_size=(3, 3), stride=1, padding=1)
        # Crop + concat step between these 2
        self.convr2 = ConvBnRelu(out_channels, out_channels, kernel_size=(3, 3), stride=1, padding=1)
     
    def _concat(self, upsampled, bypass):    
        return torch.cat((upsampled, bypass), 1)

    def forward(self, x, down_tensor):
        x = self.upSample(x)
        x = self._concat(x, down_tensor)
        x = self.convr1(x)    
        x = self.convr2(x)
        
       # x = self.convr3(x)
        return x

class StackDecoder2(nn.Module):
    def __init__(self, in_channels1,in_channels2, out_channels):
        super(StackDecoder2, self).__init__()

        self.upSample = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)
        #self.upSample = nn.ConvTranspose2d(in_channels1,in_channels1, (2,2), stride=2)
        self.convr1 = ConvBnRelu(in_channels1+in_channels2, out_channels, kernel_size=(3, 3), stride=1, padding=1)
        # Crop + concat step between these 2
        self.convr2 = ConvBnRelu(out_channels, out_channels, kernel_size=(3, 3), stride=1, padding=1)
        self.convr3 = ConvBnRelu(out_channels, out_channels, kernel_size=(3, 3), stride=1, padding=1)
     
    def _concat(self, upsampled, bypass):
        return torch.cat((upsampled, bypass), 1)

    def forward(self, x, down_tensor):
        x = self.upSample(x)
        x = self._concat(x, down_tensor)
        x = self.convr1(x)    
        x = self.convr2(x)
        x = self.convr3(x)
       # x = self.convr3(x)
        return x


class UNetOriginal(nn.Module):
    @staticmethod
    def weight_init(m):
        if isinstance(m, nn.Conv2d):
            torch.nn.init.kaiming_normal(m.weight.data)

    def __init__(self, in_shape):
        super(UNetOriginal, self).__init__()
        channels, height, width = in_shape

        self.down1 = StackEncoder(channels,16)
        self.down2 = StackEncoder(16, 16)
        self.down3 = StackEncoder2(16, 16)
        self.down4 = StackEncoder2(16, 16)
        self.down5 = StackEncoder2(16, 16)
        self.center = nn.Sequential(
            ConvBnRelu(16, 16, kernel_size=(3, 3), stride=1, padding=1),
            ConvBnRelu(16, 16, kernel_size=(3, 3), stride=1, padding=1)
        )

        self.up1 = StackDecoder2(in_channels1=16,in_channels2=16, out_channels=16)
        self.up2 = StackDecoder2(in_channels1=16,in_channels2=16, out_channels=16)
        self.up3 = StackDecoder2(in_channels1=16,in_channels2=16, out_channels=16)
        self.up4 = StackDecoder(in_channels1=16,in_channels2=16, out_channels=16)
        self.up5 = StackDecoder(in_channels1=16,in_channels2=16, out_channels=16)
        self.output_seg_map = nn.Conv2d(16, 2, kernel_size=(1, 1), padding=0, stride=1)

    def forward(self, x):
        x, x_trace1 = self.down1(x)  # Calls the forward() method of each layer
        x, x_trace2 = self.down2(x)
        x, x_trace3 = self.down3(x)
        x, x_trace4 = self.down4(x)
        x, x_trace5 = self.down5(x)

        x = self.center(x)

        x = self.up1(x, x_trace5)
        x = self.up2(x, x_trace4)
        x = self.up3(x, x_trace3)
        x = self.up4(x, x_trace2)
        x = self.up5(x, x_trace1)
        
        out = self.output_seg_map(x)
        out = torch.squeeze(out, dim=1)
        return out

The inference using the trained model is done here.  The "UNet_Lakes" contains the trained model. The input image is imported from the Google Drive. 


In [1]:
WINDOW_SIZE = (256,256) # Patch size
WINDOW_SIZE = (256,256) # Patch size
IN_CHANNELS = 13# Number of input channels (e.g. RGB)
BATCH_SIZE = 16 # Number of samples in a mini-batch

def sigmoid(z):
    return 1/(1+np.exp(-z))
def count_sliding_window(top, step=10, window_size=(20,20)):
    """ Count the number of windows in an image """
    c = 0
    for x in range(0, top.shape[0], step):
        if x + window_size[0] > top.shape[0]:
            x = top.shape[0] - window_size[0]
        for y in range(0, top.shape[1], step):
            if y + window_size[1] > top.shape[1]:
                y = top.shape[1] - window_size[1]
            c += 1
    return c

def grouper(n, iterable):
    """ Browse an iterator by chunk of n elements """
    it = iter(iterable)
    while True:
        chunk = tuple(itertools.islice(it, n))
        if not chunk:
            return
        yield chunk

def sliding_window(top, step=10, window_size=(20,20)):
    """ Slide a window_shape window across the image with a stride of step """
    for x in range(0, top.shape[0], step):
        if x + window_size[0] > top.shape[0]:
            x = top.shape[0] - window_size[0]
        for y in range(0, top.shape[1], step):
            if y + window_size[1] > top.shape[1]:
                y = top.shape[1] - window_size[1]
            yield x, y, window_size[0], window_size[1]


def get_random_pos(img, window_shape):
    """ Extract of 2D random patch of shape window_shape in the image """
    w, h = window_shape
    W, H = img.shape[-2:]
    x1 = random.randint(0, W - w - 1)
    x2 = x1 + w
    y1 = random.randint(0, H - h - 1)
    y2 = y1 + h
    return x1, x2, y1, y2
 
def sliding_window_N(top, step=10, window_size=(20,20)):
    """ Slide a window_shape window across the image with a stride of step """
    coord=[]    
    for x in range(0, top.shape[0], step):
        if x + window_size[0] > top.shape[0]:
            x = top.shape[0] - window_size[0]
        for y in range(0, top.shape[1], step):
            if y + window_size[1] > top.shape[1]:
                y = top.shape[1] - window_size[1]
            coord.append((x,y))            
    return coord

def testOnly(net, test_ids, all=False, stride=WINDOW_SIZE[0], batch_size=BATCH_SIZE, window_size=WINDOW_SIZE):
    #test_files  = glob(test_Folder)
    test_ids = list(range(1,len(test_files)+1))
    for k in range(len(test_ids)):
        print(test_files[int(test_ids[k])-1])
        with rio.open(test_files[int(test_ids[k])-1]) as img :
          test_images= img.read()
          

        test_images = test_images/16000
        all_preds = []
        net.eval()
  
        img=test_images
        pred = np.zeros((img.shape[0],img.shape[1],N_CLASSES),)
        gt = np.zeros((img.shape[0],img.shape[1]))
        stride=128
        total = count_sliding_window(gt, step=stride, window_size=window_size) // batch_size
        for i, coords in enumerate(tqdm(grouper(batch_size, sliding_window(gt, step=stride, window_size=window_size)), total=total, leave=False)):
            # Display in progress results
                    
            # Build the tensor
            image_patches = [np.copy(img[x:x+w, y:y+h]).transpose((2,0,1)) for x,y,w,h in coords]
            image_patches = np.asarray(image_patches)
            image_patches = Variable(torch.from_numpy(image_patches).cuda(), volatile=True)
            
            # Do the inference
            outs = net(image_patches)
            outs = F.softmax(outs, dim=1)
            outs = outs.data.cpu().numpy()
            
            # Fill in the results array
            for out, (x, y, w, h) in zip(outs, coords):
                out = out.transpose((1,2,0))
                pred[x:x+w, y:y+h] += out
            del(outs)

        pred = np.argmax(pred, axis=-1)
        
        fig = plt.figure()
        fig.add_subplot(1,2,1)
        plt.imshow(np.asarray(255 * img[:,:,1], dtype='uint8'))
        fig.add_subplot(1,2,2)
        plt.imshow((pred))
        plt.show()
        return pred

LABELS = ["Backgr","Lakes"] 
mylabels = np.array(LABELS)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# instantiate the network     
net = UNetOriginal((IN_CHANNELS,WINDOW_SIZE[0],WINDOW_SIZE[1]))      
net=net.cuda() 
path = F"/content/gdrive/My Drive/Nauman_Lakes_96_0948_0890"
net.load_state_dict(torch.load(path))
#net = train(net, train_loader, val_loader, test_loader, optimizer,200, scheduler)
test_files  = glob(r'/content/gdrive/My Drive/20180806T053641_20180806T054709_T43SFV.tif')
print(test_files)
test_ids=[1]
pred = testOnly(net, test_ids, all=False, stride=min(WINDOW_SIZE))
#acc = test(net, testOnly_ids, IMAGE_FOLDER_Test, BATCH_SIZE, WINDOW_SIZE, len(LABELS)) 


NameError: ignored

The predicted image along with the false color image is displayed over the openstreet map base layer using Folium library. 

In [ ]:
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'
bands=['B1','B2','B3','B4','B5','B6','B7','B8','B9']
bounds1 = [[lat1,lon1],[lat2, lon2]]
m=folium.Map(location=[ lat1,lon1],zoom_start=12)
mapid=image.getMapId({'bands':['B5','B4','B3']})    
folium.TileLayer(
  tiles=EE_TILES.format(**mapid),
  attr='Google Earth Engine',
  overlay=True,
 ).add_to(m)  
#m.add_child(folium.LayerControl())


img = folium.raster_layers.ImageOverlay(
  name='PredictedImage',
  image=pred,
  bounds=bounds1,
  interactive=True,
  overlay=True,
)
img.add_to(m)
m.add_child(folium.LayerControl())
m